# 데이터 불러오기, 라이브러리 선언  


In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [15]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 55.0 MB/s 


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import urllib.request
from collections import Counter

%matplotlib inline
import re
from konlpy.tag import Twitter
import csv

from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# 로컬에서 불러오는게 느려 구글드라이브를 통해 불러옴

recipe = pd.read_csv('/content/gdrive/MyDrive/cp1/TB_RECIPE_SEARCH.csv', encoding='CP949')

In [4]:
recipe.head()

,레시피일련번호,레시피제목,요리명,등록자ID,등록자명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리인분명,요리난이도명,요리시간명,최초등록일시
0,128671,어묵김말이,어묵김말이,skfo0701,꽃날,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,2.010000e+13
1,128892,두부에 꼬리가 달렸어요!!,NaN,skfo0701,꽃날,5538,3,26,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내,2.010000e+13
2,128932,입안에서 톡톡톡,NaN,skfo0701,꽃날,6802,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내,2.010000e+13
3,131871,★현미호두죽,현미호두죽,cds1117,햇님&별님,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,2.010000e+13
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,skfo0701,꽃날,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내,2.010000e+13


# Feature Engineering, 결측치 제거

In [5]:
# [함수] : 결측치 제거

recipe.dropna(how='any', inplace=True)

In [6]:
# 필요없는 컬럼 제거

recipe = recipe.drop(['레시피일련번호','레시피제목','등록자ID','등록자명','최초등록일시'], axis=1)
recipe.head()

,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리인분명,요리난이도명,요리시간명
0,어묵김말이,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내
3,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내
4,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내
5,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,1인분,초급,30분이내
6,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,2인분,초급,30분이내


In [8]:
# [함수] : 컬럼명 간소화

def columns_rename(data):

  result = data.rename(columns={'요리방법별명':'요리방법','요리상황별명':'요리상황','요리재료별명':'분류','요리종류별명':'요리종류','요리인분명':'요리인분','요리난이도명':'난이도','요리시간명':'요리시간'})

  return result


recipe = columns_rename(recipe)

In [9]:
# [함수] : '요리시간': 숫자로 변경 (ex. 60분이내 -> 60)

import re

def to_minute(data):

    if '시간' in data:
        result = re.sub('[^0-9]', '', data)
        result = pd.to_numeric(result) * 60
    else:
        result = re.sub('[^0-9]', '', data)

    return result


recipe['요리시간'] = recipe['요리시간'].apply(to_minute)

In [10]:
# [함수] : '난이도': 숫자로 변경 (ex. '초급' -> 2)

def difficulty_to_num(data):
  data = data.replace({'난이도': '아무나'},1)
  data = data.replace({'난이도': '초급'},2)
  data = data.replace({'난이도': '중급'},3)
  data = data.replace({'난이도': '고급'},4)
  data = data.replace({'난이도': '신의경지'},5)

  return data


recipe = difficulty_to_num(recipe)

In [11]:
# [함수] : '요리인분': 숫자로 변경 (ex. '2인분' -> 2)

def cooking_servings (data):

  data['요리인분'] = data['요리인분'].str[0]

  return data


recipe = cooking_servings(recipe)

In [13]:
print(recipe.shape)
recipe.head()

(114512, 13)


,요리명,조회수,추천수,스크랩수,요리방법,요리상황,분류,요리종류,요리소개,요리재료내용,요리인분,난이도,요리시간
0,어묵김말이,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2,2,60
3,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2,2,30
4,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2,2,60
5,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,1,2,30
6,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,2,2,30


# 재료 전처리, 토큰화

In [19]:
recipe = recipe[recipe['요리종류'] == '메인반찬']

In [20]:
stop_words = [
    "큰술"
    , "작은술"
    , "큰술"

]

pat = r'\b(?:{})\b'.format('|'.join(stop_words))

recipe['요리재료내용'] = recipe['요리재료내용'].str.replace(pat, '')

recipe[['요리재료내용']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


,요리재료내용
4,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...
6,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...
7,[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올...
29,[재료] 양배추 1/2통| 숙주 1줌| 오징어 1마리| 새우 6마리| 베이컨 2줄|...
32,[재료] 갈비 900그램| 월계수잎 3장| 통마늘 7쪽| 후춧가루 1/2큰술| 청주...
...,...
128335,[재료] 굴 400g| 달걀 4개| 쪽파 2줄기| 부추 10줄기| 홍고추 2개| 청...
128370,[연어] 연어 뱃살 200g| 흑후추 조금| 소금 조금| 올리브유 조금 [연어폰즈소...
128377,[재료] 돼지고기앞다리 불고기용 600g| 양파 1/2개| 대파 1대| 청양고추 3...
128392,[재료] 순대 400g| 청양고추 1개| 계란 3개 [양념] 소금 2꼬집| 올리브유...


In [21]:
t = Twitter()

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [22]:
def conv_pos_text(trg_txt):
    return ' '.join([w for w, _ in t.pos(trg_txt, norm=True) if _ == 'Noun'])

recipe['pos_text'] = recipe['요리재료내용'].apply(lambda x: conv_pos_text(x.replace('[재료]', '')))
recipe

,요리명,조회수,추천수,스크랩수,요리방법,요리상황,분류,요리종류,요리소개,요리재료내용,요리인분,난이도,요리시간,pos_text
4,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2,2,60,북어 포 마리 찹쌀 가루 양념 간장 설탕 물 파 마늘 참기름 깨소금 후춧가루 약간
6,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,2,2,30,건 표고버섯 개 오이 개 당근 개 양파 개 사과 쪽 외 야채 과일 녹말 물 녹말가루...
7,갈릭치킨,8148,0,38,튀김,손님접대,닭고기,메인반찬,요즘 갈릭이 대세인거 같아요~ 그래서 오늘은 제가 너무 좋아하는 닭과 마늘을 한번에...,[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올...,2,3,60,닭 마리 닭 양념 통 마늘 개 마늘 화이트와인 우유 올리브유 버터 마늘 솔트 소금 ...
29,오코노미야키,17312,6,219,부침,술안주,육류,메인반찬,술안주나 간식으로 쉽게 만들수 있는 일본요리,[재료] 양배추 1/2통| 숙주 1줌| 오징어 1마리| 새우 6마리| 베이컨 2줄|...,2,3,60,양배추 통 숙주 줌 오징어 마리 새우 마리 베이컨 줄 계란 개 실파 약간 오코노미야...
32,매운갈비찜,9561,4,26,찜,일상,육류,메인반찬,화끈화끈 매운맛에 입에서 살살 녹는 부드런 갈비찜이예요.,[재료] 갈비 900그램| 월계수잎 3장| 통마늘 7쪽| 후춧가루 1/2큰술| 청주...,4,3,120,갈비 그램 월계수 잎 장 통 마늘 쪽 후춧가루 술 청주 술 감자 개 양파 개 양념 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128335,굴전,1687,0,28,부침,명절,해물류,메인반찬,오늘은 생각보다 간단한 굴전(Guljeon Pan-fried Battered Oy...,[재료] 굴 400g| 달걀 4개| 쪽파 2줄기| 부추 10줄기| 홍고추 2개| 청...,2,2,30,굴 달걀 개 쪽파 줄기 부추 줄기 홍 고추 개 청양고추 개 맛술 술 가루 술 양념장...
128370,연어타다끼,867,0,29,굽기,일상,해물류,메인반찬,연어 뱃살 앞뒤를 가볍게 구워 속은 촉촉한 맛있는 연어타다끼를 만들어보았어요~^^...,[연어] 연어 뱃살 200g| 흑후추 조금| 소금 조금| 올리브유 조금 [연어폰즈소...,1,2,60,연어 연어 뱃살 후추 조금 소금 조금 올리브유 조금 연어 폰 소스 간장 술 물 술 ...
128377,콩나물제육볶음,44062,0,126,볶음,일상,돼지고기,메인반찬,주말저녁은 고기죠~~ 제육볶음에 콩나물을 넣어 맛있게 볶아봤어요. 콩나물 없이 그냥...,[재료] 돼지고기앞다리 불고기용 600g| 양파 1/2개| 대파 1대| 청양고추 3...,3,1,60,돼지고기 앞다리 불고기 용 양파 개 대파 청양고추 개 콩나물 줌 양념 및 소스 재료...
128392,계란순대전,1189,0,15,부침,초스피드,가공식품류,메인반찬,먹고남은 순대~를 활용한 부침개이네요 그냥먹는 순대보다 더 맛있네요ㅎ,[재료] 순대 400g| 청양고추 1개| 계란 3개 [양념] 소금 2꼬집| 올리브유...,2,1,10,순대 청양고추 개 계란 개 양념 소금 올리브유 술


In [23]:
stop_words = [
    "재료"
    , "개"
    , "장"
    , "쪽"
    , "적당"
    , "량"
    , "약간"
    , "컵"
    , "스푼"
    , "줌"
    , "숟가락"
    , "그램"
    , "마리"
    , "포기"
    , "움큼"
    , "봉지"
    , "취향"
    , "껏"
    , "단"
    , "통"
    , "킬로"
    , "그람"
    , "킬로그람"
    , "킬로그램"
    , "키로그램"
    , "키로그람"
    , "계량컵"
    , "모"
    , "판"
    , "봉"
    , "팩"
    , "중간"
    , "방울"
    , "아주"
    , "조금"
    , "도구"
    , "것"
    , "선택"
    , "사항"
    , "크기"
    , "중간"
    , "스"
    , "조금"
    , "근"
    , "숟갈"
    , "인분"
    , "용"
    , "빻"
    , "스픈"
    , "버무릴때"
    , "살짝"
    , "더욱"
    , "도씨"
    , "취행껏"
    , "끈"
    , "상자"
    , "종"
    , "겹"
    , "경우"
    , "여성"
    , "한편"
    , "노끈"
    , "특"
    , "청순"
    , "병뚜껑"
    , "방법"
    , "손가락"
    , "손질"
    , "천식"
    , "알루미늄"
    , "얌념"
    , "하나"
    , "둘"
    , "가닥"
    , "셋"
    , "넷"
    , "다섯"
    , "여섯"
    , "일곱"
    , "여덟"
    , "아홉"
    , "열"
    , "시중"
    , "부부"
    , "환절기"
    , "동서"
    , "네이버"
    , "서울"
    , "길거리"
    , "활용"
    , "산지"
    , "청화대"
    , "세종"
    , "박기준"
    , "양퍄"
    , "뮬"
    , "쫌"
    , "랏"
    , "펙"
    , "양퍄"
    , "차마"
    , "알류미늄"
    , "이빨"
    , "건방"
    , "무효"
    , "담배"
    , "쟁이"
    , "즐"
    , "쩌"
    , "의미"
    , "기초"
    , "아빠"
    , "먹음"
    , "호주"
    , "선생님"
    , "김슬"
    , "쪼끔씩"
    , "샌딩용"
    , "좀"
    , "처"
    , "오천원"
    , "더욱"
    , "의미"
    , "제거"
    , "만능"
    , "작은딸"
    , "물냉비냉"
    , "종"
    , "껏더"
    , "끈"
    , "겹"
    , "작년"
    , "제거"
    , "타입"
    , "특"
    , "항아리"
    , "머그잔"
    , "이티"
    , "처음"
    , "착"
    , "살알짝"
    , "소리"
    , "때문"
    , "퀵"
    , "트랙"
    , "얌념"
    , "동남아"
    , "영파"
    , "니아"
    , "박원형"
    , "랑어"
    , "머"
    , "맵"
    , "끼니"
    , "생략"
    , "섭씨"
    , "쪼금"
    , "사정"
    , "마음껏"
    , "터스"
    , "비법"
    , "양추"
    , "여"
    , "손질"
    , "구성"
    , "어슷썬거"
    , "애가"
    , "피방"
    , "샌드할"
    , "젖병"
    , "마음껏"
    , "렁"
    , "강추"
    , "물총"
    , "맛내기"
    , "룰"
    , "인원"
    , "비법"
    , "식해"
    , "메로나"
    , "패쓰"
    , "싹"
    , "아"
    , "명절"
    , "프리미엄"
    , "속"
    , "낼"
    , "우양"
    , "퍄"
    , "리알"
    , "차로"
    , "찜옷"
    , "팁"
    , "오브"
    , "스냅"
    , "구"
    , "뭉치"
    , "소시"
    , "백년"
    , "컥"
    , "킹"
    , "어린이"
    , "가량"
    , "횟"
    , "대충"
    , "대접"
    , "후춧"
    , "빼기"
    , "실제"
    , "듯"
    , "팸"
    , "그"
    , "군"
    , "먹기"
    , "인기"
    , "빈스"
    , "본문"
    , "마법"
    , "숫"
    , "칼라"
    , "꼬짐"
    , "간편"
    , "알팔파"
    , "거나"
    , "코팅"
    , "다른"
    , "뽈살데칠때"
    , "황금비율"
    , "보고"
    , "거나"
    , "크게"
    , "크드"
    , "개개"
    , "뉴"
    , "쥬키"
    , "씸장"
    , "빙"
    , "보조"
    , "마카"
    , "고니"
    , "특대"
    , "썸"
    , "본문"
    , "듬팩"
    , "낱개"
    , "은"
    , "기름종이"
    , "후"
    , "금"
    , "인증"
    , "바삭"
    , "가도"
    , "디아"
    , "툼"
    , "냉장"
    , "고형"
    , "제스트"
    , "캐럿"
    , "표교"
    , "법"
    , "별로"
    , "두께"
    , "된거"
    , "돔"
    , "일반"
    , "추리"
    , "버전"
    , "접시"
    , "길쭉"
    , "앙금"
    , "쭉쭉"
    , "최종"
    , "파운드"
    , "춤"
    , "헌터"
    , "우린"
    , "한복"
    , "주먹"
    , "앳젓"
    , "땅"
    , "재량"
    , "만들기"
    , "이"
    , "시디"
    , "넓이"
    , "반짝"
    , "침용"
    , "부케"
    , "준비"
    , "입"
    , "긴장"
    , "남방"
    , "동물복지"
    , "엄지손가락"
    , "솔솔"
    , "몇점"
    , "타"
    , "서너"
    , "만큼"
    , "직전"
    , "휘릭"
    , "무스"
    , "토픽"
    , "향사"
    , "물역"
    , "지퍼"
    , "톨"
    , "링"
    , "레드"
    , "입맛"
    , "중국"
    , "센티"
    , "알짜"
    , "몇장"
    , "갸"
    , "까르"
    , "줄때"
    , "압축"
    , "지급"
    , "백원"
    , "도드란내"
    , "종류"
    , "꼭"
    , "리터"
    , "춘천"
    , "부주"
    , "용량"
    , "김준현"
    , "힌즘"
    , "둑"
    , "방송"
    , "란"
    , "군데"
    , "때"
    , "빈"
    , "패추"
    , "코"
    , "은사"
    , "무전"
    , "양피"
    , "잔"
    , "지불"
    , "이상"
    , "기둥"
    , "용도"
    , "등"
    , "빈"
    , "나머지"
    , "스품"
    , "맘껏"
    , "필요시"
    , "건대"
    , "쏭쏭"
    , "티스푼"
    , "몬트리올"
    , "사이즈"
    , "리리"
    , "곱게빻"
    , "닉"
    , "냉"
    , "주부"
    , "긔"
    , "아래"
    , "로소"
    , "섯"
    , "갈릭"
    , "라이스"
    , "절단"
    , "린물"
    , "날"
    , "마토"
    , "겨울"
    , "술"
    , "슈가"
    , "외"
    , "및"
    , "양념"
    , "찜통"
    , "침용"
    , "아울렛"
    , "인의"
    , "내"
    , "사용권"
    , "샤워"
    , "찌게"
    , "냉동고"
    , "제외"
    , "패스"
    , "솔솔"
    , "돈설"
    , "노르웨이"
    , "중국"
    , "직사각형"
    , "앞"
    , "중"
    , "프로"
    , "긴장"
    , "추잡"
    , "둥지"
    , "직"
    , "건"
    , "원형"
    , "국자"
    , "미국"
    , "영국"
    , "일본"
    , "처리"
    , "옵"
    , "탁탁"
    , "동량"
    , "소수"
    , "입맛"
    , "옵"
    , "저리"
    , "높이"
    , "쏭쏭쏭"
    , "북"
    , "사용"
    , "가능"
    , "저리"
    , "암"
    , "스트링"
    , "여러가지"
    , "줄때"
    , "지급"
    , "쩍"
    , "프라임"
    , "기호"
    , "스위스"
    , "월"
    , "버"
    , "호텔"
    , "일드"
    , "카"
    , "스킨"
    , "감소"
    , "컷"
    , "양식"
    , "대형"
    , "실"
    , "숄트"
    , "팔도"
    , "강물"
    , "심"
    , "이내"
    , "망"
    , "메생"
    , "코리안더"
    , "심"
    , "델"
    , "버그"
    , "컷"
    , "때"
    , "송"
    , "오백원"
    , "메생"
    , "스트로베리"
    , "건문"
    , "이내"
    , "랜치"
    , "밖"
    , "양피"
    , "무전"
    , "택일"
    , "한수"
    , "생각"
    , "색깔"
    , "잔"
    , "초고"
    , "패추"
    , "안동"
    , "지불"
    , "탕"
    , "얼"
    , "의성"
    , "기꼬"    
    , "후르"
    , "삼각형"
    , "활"
    , "치츠"
    , "방송"
    , "깍"
    , "잔"
    , "블랙"
    , "레드"
    , "안동"
    , "믈"
    , "키"
    , "네모"
    , "상태"
    , "미용"
    , "참기믈"
    , "쏠트"
    , "미만"
    , "이하"
    , "이상"
    , "정도"
    , "개남짓"
    , "재는"
    , "체형"
    , "손톱"
    , "아빠스"
    , "수저"
    , "가을"
    , "체"
    , "선"
    , "썜"
    , "용지"
    , "일"
    , "건강"
    , "냄새"
    , "엄지"
    , "비율"
    , "애기"
    , "냉"
    , "고속"
    , "만"
    , "보통"
    , "음"
    , "음"
    , "초"
    , "목"
    , "주요"
    , "왕비"
    , "겨울"
    , "봄"
    , "여름"
    , "웅큼"
    , "숟저"
    , "해도"
    , "삼각"
    , "통계"
    , "발라"
    , "몇장"
    , "소수"
    , "적량"
    , "믈"
    , "산"
    , "참기믈"
    , "건대"
    , "포장"
    , "동생"
    , "재는"
    , "분량"
    , "취양것"
    , "제"
    , "만"
    , "닉"
    , "추가"
    , "전용"
    , "보관"
    , "애기"
    , "손"
    , "추천"
    , "김구"
    , "바로"
    , "자"
    , "김치양"
    , "홈플러스"
    , "맘"
    , "왕만"
    , "절단"
    , "슷"
    , "후주"
    , "움"
    , "둥이"
    , "머"
    , "다시"
    , "무철"
    , "똥"
    , "그라인더"
    , "태국"
    , "어탕"

]

pat = r'\b(?:{})\b'.format('|'.join(stop_words))
recipe['pos_text'] = recipe['pos_text'].str.replace(pat, '')

# df[(df['pos_text'].str.contains('재료'))]

recipe[['pos_text']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:579: FutureWarning: The default value of regex will change from True to False in a future version.


,pos_text
4,북어 포 찹쌀 가루 간장 설탕 물 파 마늘 참기름 깨소금 후춧가루
6,표고버섯 오이 당근 양파 사과 야채 과일 녹말 물 녹말가루 물 계란 노...
7,닭 닭 마늘 마늘 화이트와인 우유 올리브유 버터 마늘 솔트 소금 샐러드 드레...
29,양배추 숙주 오징어 새우 베이컨 줄 계란 실파 오코노미야키 반죽 밀가루 ...
32,갈비 월계수 잎 마늘 후춧가루 청주 감자 양파 고춧가루 대략 간...
...,...
128335,굴 달걀 쪽파 줄기 부추 줄기 홍 고추 청양고추 맛술 가루 양념장 레몬 ...
128370,연어 연어 뱃살 후추 소금 올리브유 연어 폰 소스 간장 물 레몬 즙 아가...
128377,돼지고기 앞다리 불고기 양파 대파 청양고추 콩나물 소스 고춧가루 황 ...
128392,순대 청양고추 계란 소금 올리브유


In [24]:
aniti_dup_set = set()

recipe['pos_text'] = recipe['pos_text'].apply(lambda x: x.split())

for i, sr in recipe.iterrows():
    aniti_dup_set.update(set(sr['pos_text']))

ingredients_dict = {
    0: "unknown"
}

for i, val in enumerate(aniti_dup_set):
    ingredients_dict[val] = i+1

ingredients_dict

{0: 'unknown',
 '오코노미': 1,
 '맛소금': 2,
 '어슷썬것': 3,
 '햄프': 4,
 '아구찜': 5,
 '흑마': 6,
 '빨간색': 7,
 '리전': 8,
 '오리목': 9,
 '알': 10,
 '아롱사태': 11,
 '그냥': 12,
 '두릅': 13,
 '청홍': 14,
 '큰슐': 15,
 '미나리': 16,
 '미소': 17,
 '쌀뜻': 18,
 '동파육': 19,
 '저온': 20,
 '가브릿살': 21,
 '진진': 22,
 '블루': 23,
 '날개': 24,
 '피조개': 25,
 '피스': 26,
 '요리': 27,
 '청정': 28,
 '맥앤치즈': 29,
 '살라미': 30,
 '가지': 31,
 '농축액': 32,
 '덴마크': 33,
 '닭찜': 34,
 '인물': 35,
 '조진': 36,
 '주창': 37,
 '여유': 38,
 '씀바귀': 39,
 '무액': 40,
 '점': 41,
 '묶음': 42,
 '넙적다리': 43,
 '워터': 44,
 '빠': 45,
 '러셋': 46,
 '애': 47,
 '주스': 48,
 '꼬막': 49,
 '우렁살': 50,
 '함초': 51,
 '한라봉': 52,
 '치마': 53,
 '수비드': 54,
 '너비아니': 55,
 '납짝': 56,
 '숄더랙': 57,
 '엄나무': 58,
 '등뼈': 59,
 '종지': 60,
 '라이': 61,
 '솔잎': 62,
 '쫑쫑': 63,
 '분모': 64,
 '카르': 65,
 '물의': 66,
 '캄폿': 67,
 '호사': 68,
 '순진': 69,
 '퍼': 70,
 '연': 71,
 '우스': 72,
 '케': 73,
 '캔햄': 74,
 '흙애': 75,
 '청국장': 76,
 '컵반': 77,
 '치즈떡볶이': 78,
 '갈비살': 79,
 '게란': 80,
 '차차': 81,
 '지질': 82,
 '간강': 83,
 '스트': 84,
 '백만': 85,
 '분만': 86,
 '겉잎': 87,
 '청치': 88

In [25]:
print(f"총 재료 개수 : {len(ingredients_dict.keys())}")

총 재료 개수 : 3177


In [26]:
data_version = "v.0.4"

In [ ]:
import json

with open(f"/content/drive/MyDrive/DATA/cp1/token_ingredients.{data_version}.json", "w", encoding='utf-8') as f:
    json.dump(ingredients_dict, f, ensure_ascii=False)

In [28]:
recipe['pos_text_tokenized'] = recipe['pos_text'].apply(lambda x: [ingredients_dict[_] for _ in x])

# recipe.drop(columns=['pos_text'], inplace=True)

In [ ]:
recipe

# 전처리 완료한 데이터 내보내기

In [30]:
# recipe.csv로 출력

recipe.to_csv('cp1_recipe.csv', index=False, encoding='CP949')